# 📢 Disclaimer

This notebook contains material copied verbatim from the [LlamaIndex documentation](https://www.llamaindex.ai/)  
and was created with the assistance of ChatGPT.  

It is intended for educational purposes only.  
All copyrights and credits belong to the LlamaIndex team and their respective authors.



# Branches and loops

A key feature of Workflows is their enablement of branching and looping logic, more simply and flexibly than graph-based approaches.

# Loops in workflows

To create a loop, we'll take our example `MyWorkflow` from the previous tutorial and add one new custom event type. We'll call it LoopEvent but again it can have any arbitrary name.

In [5]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
)
from llama_index.utils.workflow import draw_all_possible_flows

import random

In [3]:
class FirstEvent(Event):
    first_output: str

class SecondEvent(Event):
    second_output: str

class LoopEvent(Event):
    loop_output: str

In [13]:
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> FirstEvent | LoopEvent:
        if random.randint(0,1) == 0:
            print('Bad thing happend')
            return LoopEvent(loop_output= 'Back to step one.')
        else:
            print('Good thing happend')
            return FirstEvent(first_output = 'First step complete.')

    @step
    async def step_two(self, ev:FirstEvent) -> SecondEvent:
        print(ev.first_output)
        return SecondEvent(second_output = 'Second step complete.')     

    @step
    async def step_three(self, ev:SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result = 'Workflow complete.')     

w = MyWorkflow(timeout= 10,verbose= False)
result = await w.run(first_input = 'Start the workflow') 
print(result)

Bad thing happend
Bad thing happend
Bad thing happend
Good thing happend
First step complete.
Second step complete.
Workflow complete.


In [14]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="Loops_in_workflows.html")

Loops_in_workflows.html


gio: file:///home/tshepiso/workspace/coldblooded-agents/LlamaIndex-Tutorials/worflows/Loops_in_workflows.html: Failed to find default application for content type ‘text/html’


In [18]:
from IPython.display import HTML, display
display(HTML(open('Loops_in_workflows.html', encoding='utf-8').read()))

# Branches in workflows

Closely related to looping is branching. As you've already seen, you can conditionally return different events. Let's see a workflow that branches into two different paths:

In [24]:
class BranchA1Event(Event):
    payload: str


class BranchA2Event(Event):
    payload: str


class BranchB1Event(Event):
    payload: str


class BranchB2Event(Event):
    payload: str


class BranchWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("Go to branch A")
            return BranchA1Event(payload="Branch A")
        else:
            print("Go to branch B")
            return BranchB1Event(payload="Branch B")

    @step
    async def step_a1(self, ev: BranchA1Event) -> BranchA2Event:
        print(ev.payload)
        return BranchA2Event(payload=ev.payload)

    @step
    async def step_b1(self, ev: BranchB1Event) -> BranchB2Event:
        print(ev.payload)
        return BranchB2Event(payload=ev.payload)

    @step
    async def step_a2(self, ev: BranchA2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch A complete.")

    @step
    async def step_b2(self, ev: BranchB2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch B complete.")

In [28]:
w = BranchWorkflow(timeout= 10,verbose= False)
result = await w.run(first_input = 'Start the workflow') 
print(result)

Go to branch B
Branch B
Branch B
Branch B complete.


In [25]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(BranchWorkflow, filename="Branches_in_workflows.html")

Branches_in_workflows.html


gio: file:///home/tshepiso/workspace/coldblooded-agents/LlamaIndex-Tutorials/worflows/Branches_in_workflows.html: Failed to find default application for content type ‘text/html’


In [26]:
from IPython.display import HTML, display
display(HTML(open('Branches_in_workflows.html', encoding='utf-8').read()))